In [1]:
import pandas as pd              # A beautiful library to help us work with data as tables
import numpy as np               # So we can use number matrices. Both pandas and TensorFlow need it. 
import matplotlib.pyplot as plt  # Visualize the things
import tensorflow as tf          # Fire from the gods

In [3]:
#Step one: Load data
dataframe = pd.read_csv('data.csv') #dataframe
dataframe = dataframe.drop(['index', 'price', 'sq_price'], axis=1)
dataframe = dataframe[0:10]
dataframe

,area,bathrooms
0,2104.0,3.0
1,1600.0,3.0
2,2400.0,3.0
3,1416.0,2.0
4,3000.0,4.0
5,1985.0,4.0
6,1534.0,3.0
7,1427.0,3.0
8,1380.0,3.0
9,1494.0,3.0


In [6]:
#Step 2: Add labels
#1 is a good buy and 0 is a bad buy
dataframe.loc[:, ('y1')] = [1,1,1,0,0,1,0,1,1,1]
#y2 is the negation of y1, opposite
dataframe.loc[:, ('y2')] = dataframe['y1'] == 0
#return True/False values to 1s and 0s
dataframe.loc[:, ('y2')] = dataframe['y2'].astype(int)
dataframe

,area,bathrooms,y1,y2
0,2104.0,3.0,1,0
1,1600.0,3.0,1,0
2,2400.0,3.0,1,0
3,1416.0,2.0,0,1
4,3000.0,4.0,0,1
5,1985.0,4.0,1,0
6,1534.0,3.0,0,1
7,1427.0,3.0,1,0
8,1380.0,3.0,1,0
9,1494.0,3.0,1,0


In [9]:
#Step 3: Prepare data for tensorflow (tensors)
# tensors are generic version of vectors and matricies
# vector == 1D Tensor
# matrix == 2D Tensor
# list of list of list == 3D Tensor

#convert features to input tensor
inputX = dataframe.loc[:, ['area', 'bathrooms']].as_matrix()
#convert labels into input tensors
inputY = dataframe.loc[:, ['y1', 'y2']].as_matrix()

In [14]:
inputX

array([[  2.10400000e+03,   3.00000000e+00],
       [  1.60000000e+03,   3.00000000e+00],
       [  2.40000000e+03,   3.00000000e+00],
       [  1.41600000e+03,   2.00000000e+00],
       [  3.00000000e+03,   4.00000000e+00],
       [  1.98500000e+03,   4.00000000e+00],
       [  1.53400000e+03,   3.00000000e+00],
       [  1.42700000e+03,   3.00000000e+00],
       [  1.38000000e+03,   3.00000000e+00],
       [  1.49400000e+03,   3.00000000e+00]])

In [16]:
inputY

array([[1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0]])

In [18]:
#Step 4: Write out our hyperparameters
learning_rate = 0.000001
training_epochs = 2000 #num of iterations
display_step = 50
n_samples = inputY.size

In [30]:
#Step 5: Create our computation graph / neural network
#for feature input tensors
#palceholders are gateways for data into computation graph
x = tf.placeholder(tf.float32, [None,2]) #two features

#create weights
#2x2 float matrix, that we'll keep updating in train process
#vaiables in tf hold and update parameters
#    in memory buffers containg tensors
W = tf.Variable(tf.zeros([2,2]))

#add biases (as in f(x) = mx + b)
b = tf.Variable(tf.zeros([2]))

#multiply weights by the inputs, first calculation
#weights are are how we govern data flows in computation graph
#multiply input by weights and add biases
y_values = tf.add(tf.matmul(x, W), b)

#apply softmax to value just created
#softmax is the activation function
y = tf.nn.softmax(y_values)

#feed in matrix of labels
y_ = tf.placeholder(tf.float32, [None,2])

In [32]:
#Step 6: Perform training
#create cost function, mean squared error
#reduce sum computes the sum of elememts across dimensions of tensor
cost = tf.reduce_sum(tf.pow(y_ - y, 2))/(2*n_samples)

#gradient descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [37]:
#initialize vaiables and tensorflow session
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)


In [42]:
#training loop
for i in range(training_epochs):
    sess.run(optimizer, feed_dict={x: inputX, y_: inputY})
    
    #write out logs of training
    if (i) % display_step == 0:
        cc = sess.run(cost, feed_dict={x: inputX, y_ :inputY})
        print "Training step:", '%04d' % (i), "cost=", "{:.9f}".format(cc)
        
print "Optimization Finished"
training_cost = sess.run(cost, feed_dict={x: inputX, y_ :inputY})
print "Training cost=", training_cost,"W=", sess.run(W),"b=", sess.run(b)

Training step: 0000 cost= 0.109534547
Training step: 0050 cost= 0.109534480
Training step: 0100 cost= 0.109534405
Training step: 0150 cost= 0.109534346
Training step: 0200 cost= 0.109534286
Training step: 0250 cost= 0.109534219
Training step: 0300 cost= 0.109534144
Training step: 0350 cost= 0.109534070
Training step: 0400 cost= 0.109534003
Training step: 0450 cost= 0.109533928
Training step: 0500 cost= 0.109533869
Training step: 0550 cost= 0.109533802
Training step: 0600 cost= 0.109533742
Training step: 0650 cost= 0.109533668
Training step: 0700 cost= 0.109533593
Training step: 0750 cost= 0.109533526
Training step: 0800 cost= 0.109533451
Training step: 0850 cost= 0.109533392
Training step: 0900 cost= 0.109533310
Training step: 0950 cost= 0.109533250
Training step: 1000 cost= 0.109533176
Training step: 1050 cost= 0.109533116
Training step: 1100 cost= 0.109533049
Training step: 1150 cost= 0.109532975
Training step: 1200 cost= 0.109532915
Training step: 1250 cost= 0.109532833
Training ste

In [44]:
sess.run(y, feed_dict = {x: inputX})

array([[ 0.71120936,  0.28879055],
       [ 0.66498959,  0.33501041],
       [ 0.73651242,  0.26348755],
       [ 0.6471585 ,  0.3528415 ],
       [ 0.78329271,  0.2167073 ],
       [ 0.70070404,  0.29929596],
       [ 0.65866917,  0.34133083],
       [ 0.64830232,  0.35169765],
       [ 0.64370346,  0.35629663],
       [ 0.65481079,  0.34518918]], dtype=float32)

In [45]:
#it is saying all houses are good
#how to fix? Add another hidden layer